In [1]:
import json
import re
from neo4j.v1 import GraphDatabase, basic_auth

In [84]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth = basic_auth("neo4j", "temppassword1"))
session = driver.session()

In [2]:
cia = []
with open("../factbook.json") as infile:
    for line in infile:
        cia_fact = json.loads(line)
        cia.append(cia_fact)

In [87]:
for country in cia: #this will go through cia and plot everything
    b = country['region']
    if b == None:
        c = "None"
    else:
        b = b.replace(" ", "")
        c = re.sub('[^a-zA-Z0-9 \n\.]', '', b)
    
    name = country['countryname']
    code = country['countrycode']
    session.run("MERGE (a: "+c+" {name: {name}, code: {code}})", {"name": name, "code": code})
    
    try:  
        gdp = country['sections']['Economy'][2]['category_data'].split('(')[0] #i think some entry doesnt have brackets
        session.run("MATCH (a {name: {name}}) SET a.gdp = {gdp}", {"name": name, "gdp": gdp})
    except:
        print(" ")
    try:
        pop = country['sections']['People and Society'][4]['category_data'].split(' ')[0]
        session.run("MATCH (n {name: {name}}) SET n.population = {pop}", {"name": name, "pop": pop})
    except:
        print(" ")
    
    hasGov = 1
    govern
    try:
        govType = country['sections']['Government'][1]['category_data']
        govern = govType
        session.run("MERGE (b: GovType {type: {govType}})", {"govType": govType})
    except:
        hasGov = 0
    try:
        if(hasGov == 1):
            session.run("MATCH (a), (b)"
                       +"WHERE a.name = {name} AND b.type = {govern}"
                       +"MERGE (a)-[r:Goverment]->(b)",
                       {"name": name, "govern": govern})
    except:
        print(" ")
        
#     try:
#         dispute = country['sections']['Transnational Issues'][0]['category_data']
#         if dispute != "none":
#             session.run("MERGE (c: TransnationalIssue {type: InternatDispute})")
#             session.run("MATCH (a), (b)"
#                        +"WHERE a.name = {name} AND b.type = InternatDispute"
#                        +"MERGE (a)-[r:Is_Dispute {specific: {dispute}}]->(b)",
#                        {"name": name, "dispute": dispute})
#     except:
#         print(dispute)

    try:
        lang = country['sections']['People and Society'][2]['category_data']
        lang = "".join(lang)
        lang = lang.split(',')
        for a in lang:
            temLang = a.split(' ')
            tempLang = temLang[0]
            if temLang[0] == '':
                tempLang = temLang[1]
            session.run("MERGE (a: Language {type: {tempLang}})", {"tempLang": tempLang})
            session.run("MATCH (a), (b)"
                       +"WHERE a.name = {name} AND b.type = {tempLang}"
                       +"MERGE (a)-[r:Speaks]->(b)",
                       {"name": name, "tempLang": tempLang})
    except:
        print(" ")

In [3]:
print (cia[52])

{'map_info': {'co_locator_img_base64': 'R0lGODlhgwHtAfcAAOnp6bGxo8zMxbm5pp6eVbq6kvv7+8TEs/f398DAkLy8uaWlnaioZqmpaqSkZLOzprS0hq2tcLa2ka2tqrS0lZ+fWZycUq+vmLW1qba2jbq6or29kbm5iLOzqLW1lrGxnq6uorGxd6urbL+/nbS0fpqaTsjIyJ+fn7Ozs7m5uaysrKampqGhoby8vJubm6qqqvn5+Z2dnbW1tfPz89nZ2czMzObm5re3t7Gxsb+/v8bGxuzs7KioqKOjo66urt/f39PT08vLy+Tk5Pj4+PHx8dLS0oiILs/Pz/X19YuLM9bW1t3d3fz8/O7u7paWR+vr6+fn58nJyYKCJODg4Kurq/j48ru7u5iYS5OTQtvb28rKyqGhWpubUOjo1piYSqKiosDAwLCwgba2tpCQPcPDw9XV1dfX17KysrCwsLi4uK+vr7q6ur29va2tndHR0cHBwaCgWKysd42NOPDw5NjY2NDQrKaml7i4guDgyLCwdLCwjcfHx6mpqaeno4WFKbS0tKWlpcTExMjInq2traiooKWlobKyi76+vqqqbq2tjqysmM3Nzc7Oztzc3K6uk8LCwpCQPKSkpNjYuuHh4a+vn4mJMbCwlampcqqqd6enp6enm66umtTU1LCwnaKiX56enq2tfNDQ0Kqqm7Gxhu3t7fLy8uPj46Kin4SEJ6qqoq+viZSURKuroKyskaamZOrq6qmpnqKiXKWlabGxkpycnJmZS62tlrCwqq+vkKamZqurcqCgoKyspbGxmrKyl/T09Kiobtra2qenacXFxamppe/v47m5n5KSP7S0gdDQzLCwf6ysbo2NN7a2h8rKpa6ud+Xl5aurk6mplszMva6ugcvLsbe3qZeXSbW1mbi4mre3luXl0ri4tc/Py6urp6urjMHBlbKysMnJq97e3qWlYcXFw7KyoLKygbe3oLe3lPDw8LOz

In [11]:
print (cia[6]['sections']['Economy'][2]['category_data'].split('(')[0])

$12.8 billion 
